In [1]:
# MMoE_online_estimate_nrmse.py
import os
import time
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Concatenate, Multiply, Lambda,
                                     Dropout, BatchNormalization, GlobalAveragePooling1D,
                                     Reshape, Conv1D, MaxPooling1D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import BinaryAccuracy, SparseCategoricalAccuracy
import matplotlib.font_manager as fm

# ---------- 中文字体 ----------
def set_chinese_font():
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                      'C:/Windows/Fonts/simhei.ttf',
                      '/System/Library/Fonts/PingFang.ttc']
        for font_path in font_paths:
            if os.path.exists(font_path):
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=font_path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 未找到指定字体，使用默认兼容字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"):
    data = np.load(npz_path, allow_pickle=True)
    signals = data["signals"]
    labels = data["labels"].astype(np.int32)
    jnr_values = data["jnr_values"].astype(np.float32)
    fs = float(data["fs"])
    L = int(data["L"])
    noise_power_db = float(data["noise_power_db"])
    type2label = data["type_to_label"].item()

    interference_type_names = {
        "satellite_signal": "Satellite_Signal",
        "single_tone": "Single_Tone",
        "comb_spectra": "Comb_Spectra",
        "sweeping": "Sweeping-LFM",
        "pulse": "Pulse",
        "frequency_hopping": "Frequency_Hopping",
        "noise_fm": "Noise_FM",
        "noise_am": "Noise_AM",
        "random_combination": "Random_Combination"
    }
    label2name = {i: interference_type_names[k] for k, i in type2label.items()}
    return {
        "signals": signals,
        "labels": labels,
        "jnr_values": jnr_values,
        "fs": fs,
        "L": L,
        "noise_power_db": noise_power_db,
        "type2label": type2label,
        "label2name": label2name
    }

# ---------- 在线估算标签 ----------
def estimate_start_end(signal, fs, threshold_factor=2.0):
    power = signal ** 2
    avg_pow = np.mean(power)
    thresh = threshold_factor * avg_pow
    above = power > thresh
    diff = np.diff(above.astype(int))
    starts = np.where(diff == 1)[0]
    ends = np.where(diff == -1)[0]
    if len(starts) == 0 or len(ends) == 0:
        return 0.0, 0.0
    return float(starts[0] / fs * 1e3), float(ends[-1] / fs * 1e3)

def estimate_jnr(signal, noise_power_db):
    total_power = 10 * np.log10(np.mean(signal ** 2) + 1e-12)
    return float(total_power - noise_power_db)

# ---------- 数据预处理 ----------
def preprocess_data(dataset, aug_ratio=0.1):
    signals = dataset["signals"]
    labels = dataset["labels"]
    jnr_values = dataset["jnr_values"]
    L = dataset["L"]
    noise_power_db = dataset["noise_power_db"]

    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() for s in signals])

    no_key = "satellite_signal"
    det_labels = (labels != dataset["type2label"][no_key]).astype(np.float32)

    # 在线估算回归标签
    param_labels = []
    for sig in signals:
        st, et = estimate_start_end(sig, dataset["fs"])
        jnr = estimate_jnr(sig, noise_power_db)
        param_labels.append([st, et, jnr])
    param_labels = np.array(param_labels, dtype=np.float32)

    # 丢弃含 NaN/Inf 的样本
    mask = ~(
        np.any(np.isnan(signals), axis=1) |
        np.any(np.isinf(signals), axis=1) |
        np.isnan(det_labels) | np.isinf(det_labels) |
        np.isnan(labels) | np.isinf(labels) |
        np.any(np.isnan(param_labels), axis=1) | np.any(np.isinf(param_labels), axis=1)
    )
    print(f"🧹 丢弃 {np.sum(~mask)} / {len(mask)} 条含 NaN/Inf 的样本")
    signals, det_labels, labels, param_labels, jnr_values = \
        signals[mask], det_labels[mask], labels[mask], param_labels[mask], jnr_values[mask]

    X_train, X_tmp, y_det_train, y_det_tmp, y_type_train, y_type_tmp, y_param_train, y_param_tmp, jnr_train, jnr_tmp = train_test_split(
        signals, det_labels, labels, param_labels, jnr_values,
        test_size=0.3, random_state=42, stratify=labels
    )
    X_val, X_test, y_det_val, y_det_test, y_type_val, y_type_test, y_param_val, y_param_test, jnr_val, jnr_test = train_test_split(
        X_tmp, y_det_tmp, y_type_tmp, y_param_tmp, jnr_tmp,
        test_size=2/3, random_state=42, stratify=y_type_tmp
    )

    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_det_train": y_det_train, "y_det_val": y_det_val, "y_det_test": y_det_test,
        "y_type_train": y_type_train, "y_type_val": y_type_val, "y_type_test": y_type_test,
        "y_param_train": y_param_train, "y_param_val": y_param_val, "y_param_test": y_param_test,
        "jnr_values_train": jnr_train, "jnr_values_val": jnr_val, "jnr_values_test": jnr_test,
        "type2label": dataset["type2label"], "label2name": dataset["label2name"],
        "L": L, "fs": dataset["fs"], "noise_power_db": noise_power_db
    }

# ---------- MMoE 模型 ----------
def build_mmoe_model(input_shape, num_classes, num_experts=4, expert_dim=64, gate_dim=32):
    inputs = Input(shape=input_shape)
    x = Reshape((input_shape[0], 1))(inputs)
    
    # 共享主干
    x = Conv1D(64, 7, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(256, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    shared_features = GlobalAveragePooling1D()(x)

    # 专家网络
    experts = []
    for i in range(num_experts):
        expert = Dense(expert_dim, activation='relu', name=f'expert_{i}')(shared_features)
        expert = Dense(expert_dim, activation='relu', name=f'expert_{i}_2')(expert)
        experts.append(expert)
    experts_stack = tf.stack(experts, axis=1)

    # 门控网络
    def build_gate(input_tensor, num_experts, name):
        gate = Dense(gate_dim, activation='relu')(input_tensor)
        gate = Dense(num_experts, activation='softmax', name=name)(gate)
        return gate

    gate_det = build_gate(shared_features, num_experts, name='gate_detection')
    gate_cls = build_gate(shared_features, num_experts, name='gate_classification')
    gate_reg = build_gate(shared_features, num_experts, name='gate_regression')

    def gate_apply(gate, experts_stack):
        return tf.reduce_sum(tf.expand_dims(gate, axis=-1) * experts_stack, axis=1)

    det_feat = gate_apply(gate_det, experts_stack)
    cls_feat = gate_apply(gate_cls, experts_stack)
    reg_feat = gate_apply(gate_reg, experts_stack)

    # 任务输出
    det_out = Dense(1, activation='sigmoid', name='detection_output')(det_feat)
    cls_out = Dense(num_classes, activation='softmax', name='classification_output')(cls_feat)
    reg_feat = Dense(3, activation='linear')(reg_feat)
    reg_out = Lambda(lambda z: tf.clip_by_value(z, -1e3, 1e3), name='regression_output')(reg_feat)

    model = Model(inputs, [det_out, cls_out, reg_out])
    model.compile(
        optimizer=Adam(1e-3),
        loss={
            'detection_output': BinaryCrossentropy(),
            'classification_output': SparseCategoricalCrossentropy(),
            'regression_output': MeanSquaredError()
        },
        loss_weights={
            'detection_output': 0.8,
            'classification_output': 2.0,
            'regression_output': 0.3
        },
        metrics={
            'detection_output': BinaryAccuracy(),
            'classification_output': SparseCategoricalAccuracy(),
            'regression_output': 'mae'
        }
    )
    return model

# ---------- 单任务基准 ----------
def build_single_task_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Reshape((input_shape[0], 1))(inputs)
    x = Conv1D(64, 7, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(128, 5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(256, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    shared_features = GlobalAveragePooling1D()(x)

    det_out = Dense(1, activation='sigmoid', name='detection_output')(shared_features)
    cls_out = Dense(num_classes, activation='softmax', name='classification_output')(shared_features)
    reg_feat = Dense(3, activation='linear')(shared_features)
    reg_out = Lambda(lambda z: tf.clip_by_value(z, -1e3, 1e3), name='regression_output')(reg_feat)

    model = Model(inputs, [det_out, cls_out, reg_out])
    model.compile(
        optimizer=Adam(1e-3),
        loss={
            'detection_output': BinaryCrossentropy(),
            'classification_output': SparseCategoricalCrossentropy(),
            'regression_output': MeanSquaredError()
        },
        metrics={
            'detection_output': BinaryAccuracy(),
            'classification_output': SparseCategoricalAccuracy(),
            'regression_output': 'mae'
        }
    )
    return model

# ---------- 数据增强 ----------
@tf.function
def aug_fn(x):
    x = tf.cast(x, tf.float32)
    if tf.random.uniform([]) > 0.2:
        snr = tf.random.uniform([], 5., 25.)
        noise = tf.random.normal(tf.shape(x), dtype=tf.float32) * tf.math.reduce_std(x) * tf.cast(10.0 ** (-snr / 20.0), tf.float32)
        x = x + noise
    if tf.random.uniform([]) > 0.3:
        shift = tf.random.uniform([], -100, 100, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=0)
    if tf.random.uniform([]) > 0.3:
        scale = tf.random.uniform([], 0.7, 1.3)
        x = x * scale
    if tf.random.uniform([]) > 0.7:
        freq_shift = tf.random.uniform([], -0.1, 0.1)
        n = tf.cast(tf.shape(x)[0], tf.float32)
        x = x * tf.cos(2 * np.pi * freq_shift * tf.range(n, dtype=tf.float32))
    return x

# ---------- NRMSE ----------
def nrmse(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))
    y_range = np.max(y_true, axis=0) - np.min(y_true, axis=0)
    return rmse / (y_range + 1e-8)

# ---------- 评估 + 打印 + NRMSE ----------
def evaluate_and_print(models, data, batch=128, model_name="MMoE"):
    os.makedirs("reports", exist_ok=True)
    ds_test = tf.data.Dataset.from_tensor_slices((
        data['X_test'],
        {
            'detection_output': data['y_det_test'],
            'classification_output': data['y_type_test'],
            'regression_output': data['y_param_test']
        }
    )).batch(batch).prefetch(tf.data.AUTOTUNE)

    y_det_true, y_type_true, y_param_true = [], [], []
    y_det_pred, y_type_pred, y_param_pred = [], [], []

    for x, y in ds_test:
        y_det_true.append(y['detection_output'].numpy())
        y_type_true.append(y['classification_output'].numpy())
        y_param_true.append(y['regression_output'].numpy())

        preds = [m(x, training=False) for m in models]
        det_avg = np.mean([p[0].numpy() for p in preds], axis=0)
        cls_avg = np.mean([p[1].numpy() for p in preds], axis=0)
        reg_avg = np.mean([p[2].numpy() for p in preds], axis=0)

        y_det_pred.append(det_avg)
        y_type_pred.append(cls_avg)
        y_param_pred.append(reg_avg)

    y_det_true = np.concatenate(y_det_true, axis=0)
    y_type_true = np.concatenate(y_type_true, axis=0)
    y_param_true = np.concatenate(y_param_true, axis=0)
    y_det_pred = np.concatenate(y_det_pred, axis=0)
    y_type_pred = np.concatenate(y_type_pred, axis=0)
    y_param_pred = np.concatenate(y_param_pred, axis=0)

    # 前 5 条
    print("\n" + "="*80)
    print(f"📥 {model_name} 输入信号（前 5 条）")
    print("="*80)
    for i in range(5):
        print(f"样本 {i}: {y_param_true[i]} ...")

    print("\n" + "="*80)
    print(f"📤 {model_name} 真实 vs 预测（前 5 条）")
    print("="*80)
    for i in range(5):
        print(f"样本 {i}:")
        print(f"  det_true: {y_det_true[i]:.4f}  |  det_pred: {y_det_pred[i,0]:.4f}")
        print(f"  cls_true: {y_type_true[i]}     |  cls_pred: {np.argmax(y_type_pred[i])}")
        print(f"  reg_true: {y_param_true[i]}  |  reg_pred: {y_param_pred[i]}")

    # NRMSE
    reg_nrmse = nrmse(y_param_true, y_param_pred)
    print("\n" + "="*80)
    print(f"📈 {model_name} NRMSE 报告")
    print("="*80)
    print("参数维度: [start_time, end_time, jnr_db]")
    for i, name in enumerate(["start_time", "end_time", "jnr_db"]):
        print(f"{name}: {reg_nrmse[i]:.4f}")
    print(f"平均 NRMSE: {np.mean(reg_nrmse):.4f}")

    # 保存
    with open(f"reports/{model_name}_nrmse_report.txt", "w", encoding="utf-8") as f:
        f.write("参数维度: [start_time, end_time, jnr_db]\n")
        for i, name in enumerate(["start_time", "end_time", "jnr_db"]):
            f.write(f"{name}: {reg_nrmse[i]:.4f}\n")
        f.write(f"平均 NRMSE: {np.mean(reg_nrmse):.4f}\n")
    print(f"✅ {model_name} NRMSE 报告已写入 reports/{model_name}_nrmse_report.txt")
    return reg_nrmse

# ---------- 训练单个模型 ----------
def train_single_model(data, model_idx, model_builder, epochs=120, batch=128):
    input_shape, num_classes = (data["L"],), len(data["type2label"])
    model = model_builder(input_shape, num_classes)

    unique_labels = np.unique(data['y_type_train'])
    cls_weights = compute_class_weight('balanced', classes=unique_labels, y=data['y_type_train'])
    cls_weight_dict = {lab: float(w) for lab, w in zip(unique_labels, cls_weights)}
    sample_weights = np.array([cls_weight_dict[lab] for lab in data['y_type_train']])

    def filter_nan(*args):
        x, y = args[0], args[1]
        ok = (tf.reduce_all(tf.math.is_finite(x)) &
              tf.math.is_finite(y['detection_output']) &
              tf.reduce_all(tf.math.is_finite(y['regression_output'])))
        if len(args) == 3:
            ok &= tf.math.is_finite(args[2])
        return ok

    ds_train = tf.data.Dataset.from_tensor_slices((
        data['X_train'],
        {
            'detection_output': data['y_det_train'],
            'classification_output': data['y_type_train'],
            'regression_output': data['y_param_train']
        },
        sample_weights
    ))
    ds_train = ds_train.filter(filter_nan)
    ds_train = ds_train.map(lambda x, y, w: (aug_fn(x), y, w), num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE)

    ds_val = tf.data.Dataset.from_tensor_slices((
        data['X_val'],
        {
            'detection_output': data['y_det_val'],
            'classification_output': data['y_type_val'],
            'regression_output': data['y_param_val']
        }
    ))
    ds_val = ds_val.filter(filter_nan)
    ds_val = ds_val.batch(batch).prefetch(tf.data.AUTOTUNE)

    os.makedirs("models", exist_ok=True)
    ckpt = f"models/{model_builder.__name__}_{model_idx}.keras"
    callbacks = [
        EarlyStopping(monitor='val_classification_output_sparse_categorical_accuracy', patience=20,
                      restore_best_weights=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_classification_output_sparse_categorical_accuracy', factor=0.5,
                          patience=10, min_lr=1e-5, mode='max', verbose=1),
        ModelCheckpoint(ckpt, save_best_only=True, save_weights_only=False,
                        monitor='val_classification_output_sparse_categorical_accuracy', mode='max', verbose=1)
    ]

    print(f"\n🔥 训练第 {model_idx + 1} 个模型（{model_builder.__name__}）...")
    start_time = time.time()
    history = model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=callbacks, verbose=1)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"✅ 模型 {model_idx + 1} 训练完成，耗时 {training_time:.2f} 秒")
    return model, training_time

# ---------- 主函数 ----------
def main():
    for d in ["models", "visualizations", "reports"]:
        os.makedirs(d, exist_ok=True)

    print("="*80)
    print("🚀 开始训练 MMoE 多任务模型（在线估算标签 + NRMSE）")
    print("="*80)

    dataset = load_dataset()
    data = preprocess_data(dataset, aug_ratio=0.1)

    input_shape, num_classes = (data["L"],), len(data["type2label"])
    n_models = 3
    epochs = 120
    batch_size = 128

    # MMoE
    print("\n" + "="*50)
    print("🔹 训练 MMoE 多任务模型")
    print("="*50)
    mmoe_models, mmoe_times = [], []
    for i in range(n_models):
        model, t = train_single_model(data, i, build_mmoe_model, epochs=epochs, batch=batch_size)
        mmoe_models.append(model)
        mmoe_times.append(t)
        nrmse_mmoe = evaluate_and_print([model], data, batch=batch_size, model_name="MMoE")

    # Single-Task
    print("\n" + "="*50)
    print("🔹 训练 单任务 基准模型")
    print("="*50)
    single_models, single_times = [], []
    for i in range(n_models):
        model, t = train_single_model(data, i, build_single_task_model, epochs=epochs, batch=batch_size)
        single_models.append(model)
        single_times.append(t)
        nrmse_single = evaluate_and_print([model], data, batch=batch_size, model_name="SingleTask")

    # 保存结果
    results = {
        "mmoe": {"times": mmoe_times, "nrmse": nrmse_mmoe.tolist()},
        "single": {"times": single_times, "nrmse": nrmse_single.tolist()},
        "config": {
            "input_shape": input_shape,
            "num_classes": num_classes,
            "n_models": n_models,
            "epochs": epochs,
            "batch_size": batch_size
        }
    }
    with open("reports/training_results_nrmse.json", "w") as f:
        json.dump(results, f, indent=4)

    print("\n" + "="*50)
    print("📈 训练时间统计")
    print("="*50)
    print("MMoE 模型:")
    for i, t in enumerate(mmoe_times):
        print(f"  模型 {i+1}: {t:.2f} 秒")
    print("\n单任务 模型:")
    for i, t in enumerate(single_times):
        print(f"  模型 {i+1}: {t:.2f} 秒")

if __name__ == "__main__":
    main()

2025-10-20 10:32:37.412880: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 10:32:37.451800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 10:32:37.977705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 开始训练 MMoE 多任务模型（在线估算标签 + NRMSE）
🧹 丢弃 0 / 81000 条含 NaN/Inf 的样本

🔹 训练 MMoE 多任务模型


2025-10-20 10:33:00.319984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46590 MB memory:  -> device: 0, name: NVIDIA vGPU-48GB, pci bus id: 0000:c8:00.0, compute capability: 8.9



🔥 训练第 1 个模型（build_mmoe_model）...
Epoch 1/120


2025-10-20 10:33:01.952387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [56700]
	 [[{{node Placeholder/_2}}]]
2025-10-20 10:33:01.952639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype float and shape [56700]
	 [[{{node Placeholder/_2}}]]
2025-10-20 10:33:04.993106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-10-20 10:33:05.194948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-10-20 

...

    442/Unknown - 8s 11ms/step - loss: 6.8616 - detection_output_loss: 0.2924 - classification_output_loss: 1.3124 - regression_output_loss: 5.2568 - detection_output_binary_accuracy: 0.8561 - classification_output_sparse_categorical_accuracy: 0.5248 - regression_output_mae: 0.7875

2025-10-20 11:45:52.997842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [8100,3]
	 [[{{node Placeholder/_3}}]]



Epoch 1: val_classification_output_sparse_categorical_accuracy improved from -inf to 0.34037, saving model to models/build_single_task_model_2.keras
443/443 [==============================] - 8s 13ms/step - loss: 6.8499 - detection_output_loss: 0.2923 - classification_output_loss: 1.3123 - regression_output_loss: 5.2454 - detection_output_binary_accuracy: 0.8562 - classification_output_sparse_categorical_accuracy: 0.5248 - regression_output_mae: 0.7860 - val_loss: 2.2601 - val_detection_output_loss: 0.2928 - val_classification_output_loss: 1.9101 - val_regression_output_loss: 0.0572 - val_detection_output_binary_accuracy: 0.8889 - val_classification_output_sparse_categorical_accuracy: 0.3404 - val_regression_output_mae: 0.1872 - lr: 0.0010
Epoch 2/120
441/443 [============================>.] - ETA: 0s - loss: 1.3216 - detection_output_loss: 0.2091 - classification_output_loss: 1.0989 - regression_output_loss: 0.0137 - detection_output_binary_accuracy: 0.8927 - classification_output_sp

2025-10-20 11:57:51.026145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [16200,3]
	 [[{{node Placeholder/_3}}]]



📥 SingleTask 输入信号（前 5 条）
样本 0: [ 2.000e-04  1.022e-01 -1.000e+01] ...
样本 1: [  0.021    0.0987 -10.    ] ...
样本 2: [ 3.80e-03  9.86e-02 -1.00e+01] ...
样本 3: [ 1.00e-04  1.02e-01 -1.00e+01] ...
样本 4: [ 1.200e-03  1.017e-01 -1.000e+01] ...

📤 SingleTask 真实 vs 预测（前 5 条）
样本 0:
  det_true: 1.0000  |  det_pred: 0.9114
  cls_true: 2     |  cls_pred: 2
  reg_true: [ 2.000e-04  1.022e-01 -1.000e+01]  |  reg_pred: [-1.70163810e-04  1.02522664e-01 -9.99896526e+00]
样本 1:
  det_true: 1.0000  |  det_pred: 1.0000
  cls_true: 7     |  cls_pred: 7
  reg_true: [  0.021    0.0987 -10.    ]  |  reg_pred: [ 5.5305958e-03  9.1362849e-02 -1.0002022e+01]
样本 2:
  det_true: 1.0000  |  det_pred: 1.0000
  cls_true: 3     |  cls_pred: 3
  reg_true: [ 3.80e-03  9.86e-02 -1.00e+01]  |  reg_pred: [-9.7736716e-05  9.1821812e-02 -9.9737129e+00]
样本 3:
  det_true: 1.0000  |  det_pred: 0.9769
  cls_true: 7     |  cls_pred: 7
  reg_true: [ 1.00e-04  1.02e-01 -1.00e+01]  |  reg_pred: [-2.26944685e-04  1.00380726e-01 -9.997

In [2]:
# evaluate_mmoe_1015.py
import os
import json
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score,
                             recall_score, f1_score, mean_absolute_error)
from sklearn.preprocessing import StandardScaler
import matplotlib.font_manager as fm

# ---------------------- 中文字体 ----------------------
def set_chinese_font():
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                      'C:/Windows/Fonts/simhei.ttf',
                      '/System/Library/Fonts/PingFang.ttc']
        for font_path in font_paths:
            if os.path.exists(font_path):
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=font_path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 未找到指定字体，使用默认兼容字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------------------- 加载数据集 ----------------------
def load_dataset(npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"):
    data = np.load(npz_path, allow_pickle=True)
    signals = data["signals"]
    labels = data["labels"].astype(np.int32)
    jnr_vals = data["jnr_values"].astype(np.float32)
    fs = float(data["fs"])
    L = int(data["L"])
    metadata = data["metadata"]
    type2label = data["type_to_label"].item()
    label2type = {v: k for k, v in type2label.items()}
    type2name = data["interference_type_names"].item()
    label2name = {i: type2name[k] for k, i in type2label.items()}
    return {
        "signals": signals,
        "labels": labels,
        "jnr_values": jnr_vals,
        "fs": fs,
        "L": L,
        "metadata": metadata,
        "type2label": type2label,
        "label2name": label2name,
        "type2name": type2name
    }

# ---------------------- 数据预处理 ----------------------
def preprocess_data(dataset):
    signals = dataset["signals"]
    labels = dataset["labels"]
    jnr_values = dataset["jnr_values"]
    L = dataset["L"]

    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() for s in signals])

    no_key = "satellite_signal"
    det_labels = (labels != dataset["type2label"][no_key]).astype(np.float32)

    param_labels = []
    for m in dataset["metadata"]:
        p = m.get("params", {})
        param_labels.append([
            float(p.get("start_time", 0)),
            float(p.get("end_time", 0)),
            float(p.get("jnr_db", 0))
        ])
    param_labels = np.array(param_labels, dtype=np.float32)

    X_test = signals
    y_det_test = det_labels
    y_type_test = labels
    y_param_test = param_labels
    jnr_test = jnr_values
    label2name = dataset["label2name"]

    return {
        "X_test": X_test,
        "y_det_test": y_det_test,
        "y_type_test": y_type_test,
        "y_param_test": y_param_test,
        "jnr_values_test": jnr_test,
        "label2name": label2name,
        "L": L
    }

# ---------------------- 加载模型并预测 ----------------------
def load_and_predict(model_paths, X_test):
    all_det, all_cls, all_reg = [], [], []
    for path in model_paths:
        print(f"🔁 加载模型: {path}")
        model = tf.keras.models.load_model(path)  # MMoE 无需 custom_objects
        start = time.time()
        det, cls, reg = model.predict(X_test, verbose=0)
        print(f"预测耗时: {time.time()-start:.2f} 秒")
        all_det.append(det)
        all_cls.append(cls)
        all_reg.append(reg)
    avg_det = np.mean(all_det, axis=0) > 0.5
    avg_cls = np.argmax(np.mean(all_cls, axis=0), axis=1)
    avg_reg = np.mean(all_reg, axis=0)
    return avg_det, avg_cls, avg_reg

# ---------------------- 绘制混淆矩阵 ----------------------
def plot_confusion_matrix(cm, labels, title, xlabel, ylabel, filename, dpi=150, rotate_x=False):
    cm_norm = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_norm = np.nan_to_num(cm_norm)
    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
                     xticklabels=labels, yticklabels=labels, square=True,
                     annot_kws={"size": 14})
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)
    plt.title(title, pad=20, fontsize=18)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(rotation=45 if rotate_x else 0, ha='right' if rotate_x else 'center', fontsize=14)
    plt.yticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi)
    plt.close()

# ---------------------- 按列归一化 NRMSE ----------------------
def nrmse_columnwise(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))
    y_range = np.max(y_true, axis=0) - np.min(y_true, axis=0)
    return rmse / (y_range + 1e-8)

# ---------------------- 主评估 ----------------------
def evaluate(models_dir="models",
             npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz",
             wanted_jnr=np.arange(-10, 31, 5),
             dpi=150):
    os.makedirs("visualizations", exist_ok=True)
    os.makedirs("reports", exist_ok=True)

    dataset = load_dataset(npz_path)
    data = preprocess_data(dataset)

    X_test = data["X_test"]
    y_det_test = data["y_det_test"]
    y_type_test = data["y_type_test"]
    y_param_test = data["y_param_test"]
    jnr_test = data["jnr_values_test"]
    label2name = data["label2name"]

    model_paths = [os.path.join(models_dir, f"build_mmoe_model_{i}.keras") for i in range(3)]
    avg_det, avg_cls, avg_reg = load_and_predict(model_paths, X_test)

    # 清理无效样本
    invalid_mask = (
        np.any(np.isnan(y_param_test), axis=1) |
        np.any(np.isinf(y_param_test), axis=1) |
        np.any(np.isnan(avg_reg), axis=1) |
        np.any(np.isinf(avg_reg), axis=1)
    )
    valid_y_param = y_param_test[~invalid_mask]
    valid_avg_reg = avg_reg[~invalid_mask]

    # 1. 检测混淆矩阵
    cm_det = confusion_matrix(y_det_test, avg_det)
    plot_confusion_matrix(cm_det, ['No Interference', 'Interference'],
                          'MMoE Detection Confusion Matrix',
                          'Predicted', 'True',
                          'visualizations/MMoE_detection_confusion_matrix_1015.png',
                          dpi=dpi)

    # 2. 分类混淆矩阵
    cm_type = confusion_matrix(y_type_test, avg_cls)
    plot_confusion_matrix(cm_type,
                          [label2name[i] for i in sorted(label2name.keys())],
                          'MMoE Classification Confusion Matrix',
                          'Predicted', 'True',
                          'visualizations/MMoE_classification_confusion_matrix_1015.png',
                          dpi=dpi, rotate_x=True)

    # 3. JNR vs Accuracy
    jnr_acc = []
    for jnr in wanted_jnr:
        mask = (jnr_test == jnr) & (~invalid_mask)
        acc = np.nan if mask.sum() == 0 else accuracy_score(y_type_test[mask], avg_cls[mask])
        jnr_acc.append(acc)

    plt.figure(figsize=(8, 5))
    valid_mask = ~np.isnan(jnr_acc)
    plt.plot(wanted_jnr[valid_mask], np.array(jnr_acc)[valid_mask], marker='o', linewidth=2)
    if np.any(~valid_mask):
        plt.scatter(wanted_jnr[~valid_mask], [1.0] * np.sum(~valid_mask),
                    facecolors='none', edgecolors='r', s=60)
    plt.xlabel('JNR (dB)', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('Classification Accuracy vs JNR', fontsize=16)
    plt.xticks(wanted_jnr)
    plt.grid(True, ls='--', alpha=0.5)
    plt.ylim(0, 1.05)
    plt.tight_layout()
    plt.savefig('visualizations/MMoE_jnr_vs_accuracy_1015.png', dpi=dpi)
    plt.close()

    # 4. 指标计算
    det_acc = accuracy_score(y_det_test, avg_det)
    cls_acc = accuracy_score(y_type_test, avg_cls)
    cls_precision = precision_score(y_type_test, avg_cls, average='weighted', zero_division=0)
    cls_recall = recall_score(y_type_test, avg_cls, average='weighted', zero_division=0)
    cls_f1 = f1_score(y_type_test, avg_cls, average='weighted', zero_division=0)

    param_mae = mean_absolute_error(valid_y_param, valid_avg_reg, multioutput='raw_values')
    param_nrmse = nrmse_columnwise(valid_y_param, valid_avg_reg)
    param_names = ['Start Time (ms)', 'End Time (ms)', 'JNR (dB)']

    print("\n" + "="*50)
    print("📊 MMoE 模型集成评估结果")
    print("="*50)
    print(f"检测准确率: {det_acc:.4f}")
    print(f"分类准确率: {cls_acc:.4f}")
    print(f"分类精确率: {cls_precision:.4f}, 召回率: {cls_recall:.4f}, F1: {cls_f1:.4f}")
    print("\n参数估计误差（MAE & 按列 NRMSE）:")
    for i, (name, mae, nrmse) in enumerate(zip(param_names, param_mae, param_nrmse)):
        print(f"  {name}: MAE = {mae:.4f}, NRMSE = {nrmse:.4f}")
    print(f"  平均 NRMSE（三列分别归一化） = {np.mean(param_nrmse):.4f}")
    print("\nJNR 准确率:")
    for j, acc in zip(wanted_jnr, jnr_acc):
        print(f"  {int(j)}dB: {acc if not np.isnan(acc) else 'N/A'}")

    # 5. 保存报告
    report = {
        "detection_accuracy": float(det_acc),
        "classification_accuracy": float(cls_acc),
        "classification_precision": float(cls_precision),
        "classification_recall": float(cls_recall),
        "classification_f1": float(cls_f1),
        "parameter_mae": [float(m) for m in param_mae],
        "parameter_nrmse": [float(n) for n in param_nrmse],
        "average_columnwise_nrmse": float(np.mean(param_nrmse)),
        "parameter_details": {
            param_names[i]: {"mae": float(param_mae[i]), "nrmse": float(param_nrmse[i])}
            for i in range(len(param_names))
        },
        "jnr_accuracies": {f"{int(j)}dB": float(acc) if not np.isnan(acc) else None for j, acc in zip(wanted_jnr, jnr_acc)},
        "invalid_samples_count": int(np.sum(invalid_mask))
    }
    with open("reports/MMoE_evaluation_report_1015.json", "w", encoding='utf-8') as f:
        json.dump(report, f, indent=4, ensure_ascii=False)

    print("\n✅ 评估完成！混淆矩阵与报告已保存。")

# ---------------------- 主入口 ----------------------
if __name__ == "__main__":
    evaluate(models_dir="models",
             npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz",
             wanted_jnr=np.arange(-10, 31, 5),
             dpi=150)

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🔁 加载模型: models/build_mmoe_model_0.keras
预测耗时: 10.20 秒
🔁 加载模型: models/build_mmoe_model_1.keras
预测耗时: 10.25 秒
🔁 加载模型: models/build_mmoe_model_2.keras
预测耗时: 8.70 秒

📊 MMoE 模型集成评估结果
检测准确率: 0.9754
分类准确率: 0.8876
分类精确率: 0.8918, 召回率: 0.8876, F1: 0.8872

参数估计误差（MAE & 按列 NRMSE）:
  Start Time (ms): MAE = 0.1328, NRMSE = 0.2232
  End Time (ms): MAE = 0.7672, NRMSE = 0.8073
  JNR (dB): MAE = 20.0017, NRMSE = 0.5952
  平均 NRMSE（三列分别归一化） = 0.5419

JNR 准确率:
  -10dB: 0.556
  -5dB: 0.70775
  0dB: 0.840625
  5dB: 0.89225
  10dB: 0.942
  15dB: 0.971
  20dB: 0.986875
  25dB: 0.987375
  30dB: 0.988875

✅ 评估完成！混淆矩阵与报告已保存。


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🔁 加载模型: models/build_mmoe_model_0.keras
预测耗时: 10.20 秒
🔁 加载模型: models/build_mmoe_model_1.keras
预测耗时: 10.25 秒
🔁 加载模型: models/build_mmoe_model_2.keras
预测耗时: 8.70 秒

==================================================
📊 MMoE 模型集成评估结果
==================================================
检测准确率: 0.9754
分类准确率: 0.8876
分类精确率: 0.8918, 召回率: 0.8876, F1: 0.8872

参数估计误差（MAE & 按列 NRMSE）:
  Start Time (ms): MAE = 0.1328, NRMSE = 0.2232
  End Time (ms): MAE = 0.7672, NRMSE = 0.8073
  JNR (dB): MAE = 20.0017, NRMSE = 0.5952
  平均 NRMSE（三列分别归一化） = 0.5419

JNR 准确率:
  -10dB: 0.556
  -5dB: 0.70775
  0dB: 0.840625
  5dB: 0.89225
  10dB: 0.942
  15dB: 0.971
  20dB: 0.986875
  25dB: 0.987375
  30dB: 0.988875